In [1]:
import warnings
warnings.filterwarnings("ignore") # Ignoring any warnings.
import pandas as pd
import numpy as np 
import json

def load_data(file_path):
   with open(file_path, 'r', encoding='utf-8') as file:
        skipped_lines = 0  # Counter for skipped lines due to errors
        for line_number, line in enumerate(file, 1):
            try:
                yield json.loads(line)
            except json.JSONDecodeError:
                skipped_lines += 1  # Increment the counter instead of printing
                continue
        if skipped_lines > 0:
            print(f"Completed loading. Skipped {skipped_lines} lines due to errors.")

In [2]:
raw_df=load_data('../corona-out-3')
df_ = pd.DataFrame(raw_df)

Completed loading. Skipped 101916 lines due to errors.


In [3]:
from datetime import datetime

def convert_time(dtime):
    return datetime.strftime(datetime.strptime(dtime,'%a %b %d %H:%M:%S +0000 %Y'), '%Y-%m-%d %H:%M:%S')

df_['created_at'] = df_['created_at'].apply(lambda row: convert_time(row))

In [4]:
df_.fillna('NA',inplace=True)

In [5]:
retweeted_tweets=df_.loc[df_['retweeted_status'] != 'NA']

In [6]:
og_tweets = df_[~df_['id'].isin(retweeted_tweets['id'])]

In [7]:
retweeted_tweets.dtypes

created_at                   object
id                            int64
id_str                       object
text                         object
display_text_range           object
source                       object
truncated                      bool
in_reply_to_status_id        object
in_reply_to_status_id_str    object
in_reply_to_user_id          object
in_reply_to_user_id_str      object
in_reply_to_screen_name      object
user                         object
geo                          object
coordinates                  object
place                        object
contributors                 object
is_quote_status                bool
quote_count                   int64
reply_count                   int64
retweet_count                 int64
favorite_count                int64
entities                     object
extended_entities            object
favorited                      bool
retweeted                      bool
possibly_sensitive           object
filter_level                

In [8]:
def process_func(row, og_tweet_df, user_df, retweet_df, user_data, user_id_col, final_cols_tweet):
    
    retweet_data = {key: row[key] for key in final_cols_tweet}
    retweet_data[user_id_col] = row['user']['id_str']
    og_tweet_data =  {key: row['retweeted_status'][key] for key in row['retweeted_status']}
    og_tweet_data['is_retweet'] = False
    og_tweet_data['original_tweet_id'] = None

    retweet_data['original_tweet_id'] = og_tweet_data['id_str']
    retweet_data['is_retweet'] = True

    og_user_data = {key: row['retweeted_status']['user'][key] for key in row['retweeted_status']['user']}

    og_tweet_data[user_id_col] = og_user_data['id_str']

    retweet_user_data = {key: row['user'][key] for key in user_data}

    og_tweet_df = pd.concat([og_tweet_df, pd.DataFrame([og_tweet_data])], ignore_index=True)
    user_df = pd.concat([user_df, pd.DataFrame([retweet_user_data])], ignore_index=True)
    user_df = pd.concat([user_df, pd.DataFrame([og_user_data])], ignore_index=True)

    retweet_df = pd.concat([retweet_df, pd.DataFrame([retweet_data])], ignore_index=True)

    return og_tweet_df, user_df, retweet_df

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


from joblib import Parallel, delayed
from tqdm import tqdm

user_id_col = "user_id"

final_cols_tweet = [
    "id",
    "text",
    "created_at",
    "retweet_count",
    "reply_count",
    "favorite_count",
    "lang",
    "source",
]

user_data = [
    "id",
    "name",
    "screen_name",
    "location",
    "url",
    "followers_count",
    "friends_count",
    "statuses_count",
    "favourites_count",
    "verified",
    "created_at",
    "profile_image_url",
    "geo_enabled",
]


retweet_df = pd.DataFrame(columns=final_cols_tweet)

user_df = pd.DataFrame(columns=user_data)

og_tweet_df = pd.DataFrame(columns=final_cols_tweet)

og_tweet_df, user_df, retweet_df = zip(*Parallel(n_jobs=-1)(
    delayed(process_func)(
        row,
        og_tweet_df,
        user_df,
        retweet_df,
        user_data,
        user_id_col,
        final_cols_tweet,
    )
    for idx, row in tqdm(retweeted_tweets.iterrows())
))

In [10]:
retweet_processed = pd.DataFrame()
og_tweet_processed = pd.DataFrame()
user_processed = pd.DataFrame()

def convert_to_df(i,new_df):
    new_df = pd.concat([new_df, pd.DataFrame(i)], ignore_index=True)  
    return new_df

retweet_df_new = Parallel(n_jobs=-1)(delayed(convert_to_df)(i,retweet_processed) for i in retweet_df)
retweet_df_new = pd.concat(retweet_df_new)

In [11]:
og_tweet_new = Parallel(n_jobs=-1)(delayed(convert_to_df)(i,og_tweet_processed) for i in og_tweet_df)
user_df_new = Parallel(n_jobs=-1)(delayed(convert_to_df)(i,user_processed) for i in user_df)

In [12]:
og_tweet_new = pd.concat(og_tweet_new)
user_df_new = pd.concat(user_df_new)

In [13]:
tweet_final_cols = [
    'id',
    'text',
    'created_at',
    'user_id',
    'hashtags',
    'urls',
    'retweet_count',
    'reply_count',
    'favorite_count',
    'is_retweet',
    'original_tweet_id',
    'lang',
    'source'
]

user_final_cols = [
    'id',
    'name',
    'screen_name',
    'location',
    'url',
    'followers_count',
    'friends_count',
    'statuses_count',
    'favourites_count',
    'verified',
    'created_at',
    'profile_image_url',
    'geo_enabled',
]

In [14]:
user_df_new.columns

Index(['id', 'name', 'screen_name', 'location', 'url', 'followers_count',
       'friends_count', 'statuses_count', 'favourites_count', 'verified',
       'created_at', 'profile_image_url', 'geo_enabled', 'id_str',
       'description', 'translator_type', 'protected', 'listed_count',
       'utc_offset', 'time_zone', 'lang', 'contributors_enabled',
       'is_translator', 'profile_background_color',
       'profile_background_image_url', 'profile_background_image_url_https',
       'profile_background_tile', 'profile_link_color',
       'profile_sidebar_border_color', 'profile_sidebar_fill_color',
       'profile_text_color', 'profile_use_background_image',
       'profile_image_url_https', 'profile_banner_url', 'default_profile',
       'default_profile_image', 'following', 'follow_request_sent',
       'notifications'],
      dtype='object')

In [15]:
user_df_new = user_df_new[user_final_cols]
user_df_new = user_df_new.drop_duplicates().fillna('NA')
user_df_new['created_at'] = user_df_new['created_at'].apply(lambda row: convert_time(row))
user_df_new.to_csv('./user_data.csv',index=False)

In [16]:
def get_metainfo(row):
    
    hashtags_retweet = row['entities']['hashtags']
    urls_retweet = row['entities']['urls']

    hashtags_og = row['retweeted_status']['entities']['hashtags']
    urls_og = row['retweeted_status']['entities']['urls']

    return hashtags_retweet, urls_retweet, hashtags_og, urls_og

hashtags_retweet, urls_retweet, hashtags_og, urls_og = zip(*Parallel(n_jobs=-1)(
    delayed(get_metainfo)(
        row
    )
    for idx, row in tqdm(retweeted_tweets.iterrows())
))

61101it [00:06, 9301.05it/s] 


In [17]:
def clean_tags(tags) :
    hashtags_og_ = []
    for i in tags :
        if len(i)>0:
            hashtags_og_.append([','.join(j['text'] for j in i )])
        else :
            hashtags_og_.append([])
            
    return hashtags_og_

In [18]:
def clean_urls(urls) :
    urls_og_ = []
    for i in urls :
        if len(i)>0:
            urls_og_.append([','.join(j['url'] for j in i )])
        else :
            urls_og_.append([])
    
    return urls_og_

In [19]:
retweet_df_new['hashtags'] = clean_tags(hashtags_retweet)
retweet_df_new['urls'] = clean_urls(urls_retweet)

og_tweet_new['hashtags'] = clean_tags(hashtags_og)
og_tweet_new['urls'] = clean_urls(urls_og)

In [20]:
og_tweet_new_ = og_tweet_new[tweet_final_cols]
og_tweet_new_.head(2)

,id,text,created_at,user_id,hashtags,urls,retweet_count,reply_count,favorite_count,is_retweet,original_tweet_id,lang,source
0,1253949413191344128,India’s war with Corona is ongoing.\n\nPlay yo...,Sat Apr 25 07:30:12 +0000 2020,207809313,[],[https://t.co/lKhbsMFpIA],340,101,1870,False,None,en,"<a href=""https://mobile.twitter.com"" rel=""nofo..."
0,1252576316135739392,@ozkan_yalim @DurmusYillmaz \nAçık kapalı görü...,Tue Apr 21 12:34:00 +0000 2020,1206650133976408064,[],[https://t.co/vAhzxTppCA],72,1,83,False,None,tr,"<a href=""http://twitter.com/download/android"" ..."


In [21]:
final_tweet_df = pd.concat([retweet_df_new, og_tweet_new_], ignore_index=True)
final_tweet_df.fillna('NA',inplace=True)
final_tweet_df = final_tweet_df.iloc[final_tweet_df.astype(str).drop_duplicates().index]
final_tweet_df.shape
final_tweet_df.to_csv('./tweet_data_final.csv',index=False)

In [22]:
retweeted_tweets.iloc[0]

created_at                                                 2020-04-25 12:21:42
id                                                         1254022772558368768
id_str                                                     1254022772558368768
text                         RT @BJP4India: India’s war with Corona is ongo...
display_text_range                                                          NA
source                       <a href="http://twitter.com/download/android" ...
truncated                                                                False
in_reply_to_status_id                                                       NA
in_reply_to_status_id_str                                                   NA
in_reply_to_user_id                                                         NA
in_reply_to_user_id_str                                                     NA
in_reply_to_screen_name                                                     NA
user                         {'id': 9083264927187640

In [23]:
retweeted_tweets['retweeted_status'].iloc[3]

{'created_at': 'Sat Apr 25 11:34:19 +0000 2020',
 'id': 1254010851142569984,
 'id_str': '1254010851142569984',
 'text': 'Morti COL Corona non PER il Corona.\n\nCONTE PER OTTENERE APPOGGIO VDL AL GOLPE DI AGOSTO 2019 HA CONSEGNATO (MANI E… https://t.co/3rWieZAm2G',
 'display_text_range': [0, 140],
 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
 'truncated': True,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 761787475,
  'id_str': '761787475',
  'name': 'Maurizio Gustinicchi',
  'screen_name': 'gustinicchi',
  'location': 'www.scenarieconomici.it',
  'url': None,
  'description': 'Consulenza e Amministrazione azienda - \n\nAutore di http://www.scenarieconomici.it - \nOpinionista economico di Canale Italia',
  'translator_type': 'none',
  'protected': False,
  'verified': False,
  'followers_count': 134

In [24]:
Tweetdata = pd.read_csv('/Users/ojassharma/Desktop/RUTGERS 1/SEM-2/DBMS/MAJOR PROJECT/FINAL DATA/tweet_data_final.csv')
Tweetdata.sample(n=10)

,id,text,created_at,retweet_count,reply_count,favorite_count,lang,source,user_id,original_tweet_id,is_retweet,hashtags,urls
109797,1254027175122153479,"In Illinois, liberal politicians cut sweethear...",Sat Apr 25 12:39:11 +0000 2020,387,94,723,en,"<a href=""http://twitter.com/download/iphone"" r...",278145569,NaN,False,[],['https://t.co/02Xcyu4uaw']
34440,1254044186325798912,RT @benwikler: Milwaukee’s health commissioner...,2020-04-25 13:46:47,0,0,0,en,"<a href=""http://twitter.com/download/iphone"" r...",867157331494268929,1.254030e+18,True,[],['https://t.co/fGIsLKzTkm']
36906,1254045688758927360,RT @aajtak: देश में #coronavirus के मरीजों की ...,2020-04-25 13:52:45,0,0,0,hi,"<a href=""http://twitter.com/download/android"" ...",836566509552029696,1.254020e+18,True,['coronavirus'],['https://t.co/bwwwIm3dpD']
89376,1253980381289893889,Hala anlatamadık\n1-Aşı bulunmadıkça Corona bi...,Sat Apr 25 09:33:15 +0000 2020,759,119,2716,tr,"<a href=""http://twitter.com/download/android"" ...",96547436,NaN,False,[],['https://t.co/0QCHOJxPiS']
61972,1253999190411182080,"Hallo liebe Freunde, ich werde im Moment oft g...",Sat Apr 25 10:47:59 +0000 2020,1,0,24,de,"<a href=""http://twitter.com/download/iphone"" r...",995587140707868672,NaN,False,[],['https://t.co/zCvOZJkbcd']
18171,1254034440386342913,RT @GuRuThalaiva: Thalapathy fans from Sivakas...,2020-04-25 13:08:04,0,0,0,en,"<a href=""https://mobile.twitter.com"" rel=""nofo...",53323859,1.254033e+18,True,[],[]
30546,1254041805206814720,RT @ulfposh: auweia\n (@sciforfuture = 🤡),2020-04-25 13:37:19,0,0,0,de,"<a href=""http://twitter.com/download/android"" ...",1032689451489746946,1.254034e+18,True,[],[]
43163,1254049439318147072,RT @aksually: Corona? cancelled https://t.co/X...,2020-04-25 14:07:40,0,0,0,en,"<a href=""http://twitter.com/download/android"" ...",1143001024992567303,1.253889e+18,True,[],[]
118472,1253947162490277888,Depuis corona a commencé mon oncle le matin il...,Sat Apr 25 07:21:15 +0000 2020,2,1,14,fr,"<a href=""http://twitter.com/download/iphone"" r...",439076207,NaN,False,[],['https://t.co/06mZlWJX9S']
50809,1254054007921934337,RT @brithume: And where is the evidence that C...,2020-04-25 14:25:49,0,0,0,en,"<a href=""https://mobile.twitter.com"" rel=""nofo...",47036166,1.254009e+18,True,[],['https://t.co/YPcJXU1uqw']


In [25]:
Userdata = pd.read_csv('/Users/ojassharma/Desktop/RUTGERS 1/SEM-2/DBMS/MAJOR PROJECT/FINAL DATA/user_data.csv')
Userdata.sample(n=10)

,id,name,screen_name,location,url,followers_count,friends_count,statuses_count,favourites_count,verified,created_at,profile_image_url,geo_enabled
14295,59657794,natascha artworx,nat_worx_,"Hamburg,Germany,World",https://www.patreon.com/nataschaartworx,958,257,35457,16423,False,2009-07-24 01:56:46,http://pbs.twimg.com/profile_images/8131675287...,False
71793,705781993506787328,Suresh Bhusari,suresh_bhusari,"New Delhi, India",NaN,2496,4727,33839,19493,False,2016-03-04 15:48:34,http://pbs.twimg.com/profile_images/9062103212...,False
29561,1239878996881313792,MADMEN™️,oathbraker,NaN,NaN,51,142,271,124,False,2020-03-17 11:39:30,http://pbs.twimg.com/profile_images/1253749303...,False
59260,810562650082070532,Magda M,modliszka30,"Piaseczno, Polska 🇵🇱",NaN,8455,8287,23927,95450,False,2016-12-18 19:09:30,http://pbs.twimg.com/profile_images/8756433810...,True
75680,1105341141375840257,Utsav Kumar Jaiswal,UtsavKumarJais1,"Bhubaneshwar, India",NaN,158,337,10938,11661,False,2019-03-12 05:33:59,http://pbs.twimg.com/profile_images/1248210298...,True
29506,1595762684,lizandro clavel 🇲🇽,LizandroClavel,"Santiago Pinotepa Nacional, Oa",NaN,119,208,3357,12434,False,2013-07-15 12:40:27,http://pbs.twimg.com/profile_images/1252662980...,True
7125,45261768,Srivatsa,srivatsayb,Bengaluru,http://t.me/srivatsayb,81333,2568,81908,66665,True,2009-06-07 02:00:29,http://pbs.twimg.com/profile_images/1231230013...,True
26202,25388119,HSArbeidsvoorwaarden 🇮🇹🤝🇳🇱,HenkStrating,Schoonhoven-Oude Stad,http://hsarbeidsvoorwaarden.nl,2122,1817,62005,35050,False,2009-03-19 21:21:24,http://pbs.twimg.com/profile_images/1242167615...,True
56326,609563419,Jaqueline Goldsmith USA EL CUBREBOCAS PORFIS,JaquecadeOro,Mexico DF,NaN,42218,382,5278,6384,False,2012-06-16 00:39:58,http://pbs.twimg.com/profile_images/1156033492...,False
42921,1243644535730814977,Osman Keymanlar06,Osman789p,Toprak,NaN,18,215,7045,4028,False,2020-03-27 21:02:33,http://pbs.twimg.com/profile_images/1251816219...,False
